In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle 

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#! kaggle datasets list 

In [ ]:
!kaggle datasets download -d awsaf49/coco-2017-dataset --unzip 

In [ ]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pytorch_lightning as pl
import pandas as pd
import cv2
import os 
import torchvision
from torch import nn
from torch.utils.data import Dataset ,DataLoader, random_split
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import torchmetrics
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torchvision.utils import make_grid
import math
import torch.nn.functional as F
from torchvision.ops import box_convert

In [ ]:
from PIL import Image
from pycocotools.coco import COCO
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
class myOwnDataset(Dataset):
  def __init__(self, root, annotation, transforms=None):
    self.root = root
    self.transforms = transforms
    self.coco = COCO(annotation)
    self.ids = list(sorted(self.coco.imgs.keys()))

  def __getitem__(self, idx):
    # own coco file
    coco = self.coco
    # image ID
    img_id = self.ids[idx]
    # List: get annotation id from coco
    ann_ids = coco.getAnnIds(imgIds=img_id)
    # Dictionary: target coco_annotation file for an image
    coco_annotation = coco.loadAnns(ann_ids)
    # path for input image
    path = coco.loadImgs(img_id)[0]["file_name"]
    # open the input image
    img = Image.open(os.path.join(self.root, path))
    # number of objects in the image
    num_objs = len(coco_annotation)

    # bounding boxes for objects 
    # In coco format, bbox = [xmin, ymin, width, height]
    # In pytorch, the input should be [xmin, ymin, xmax, ymax]
    boxes = []
    for i in range(num_objs):
      xmin = coco_annotation[i]["bbox"][0]
      ymin = coco_annotation[i]["bbox"][1]
      width = coco_annotation[i]["bbox"][2]
      height = coco_annotation[i]["bbox"][3]
      boxes.append([xmin, ymin, width, height])
    boxes = torch.Tensor(boxes)
    # convert box format from "xywh" to "xyxy"
    boxes = box_convert(
                boxes, in_fmt="xywh", out_fmt="xyxy"
            )
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    # labels (In my case, I only one class: target class or background)
    labels = torch.ones((num_objs, ), dtype=torch.int64)
    # tensorize img_id
    img_id = torch.tensor([img_id])
    # size of bbox (rectangular)
    areas = []
    for i in range(num_objs):
      areas.append(coco_annotation[i]["area"])
    areas = torch.as_tensor(areas, dtype=torch.float32)
    # iscrowd
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

    # Annotation is in dictionary format
    my_annotation = {}
    my_annotation["boxes"] = boxes
    my_annotation["labels"] = labels
    my_annotation["image_id"] = img_id
    my_annotation["area"] = areas
    my_annotation["iscrowd"] = iscrowd

    if self.transforms is not None:
      img = self.transforms(img)

    return img, my_annotation

  def __len__(self):
    return len(self.ids)

In [ ]:

# collate_fn needs for batch
def collate_fn(batch):
  return tuple(zip(*batch))

In [ ]:
def get_model_instance_segmentation(num_classes):
  # load instance segmentation model pre-trained on COCO
  weights = models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT
  model = models.detection.fasterrcnn_resnet50_fpn(weights=weights)
  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # replace the pre-trained head with a new one
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

  return model

In [ ]:
# path to my data and coco file
train_data_dir = "/content/coco2017/train2017"
test_data_dir = "/content/coco2017/test2017"
val_data_dir = "/content/coco2017/val2017"
train_coco = "/content/coco2017/annotations/instances_train2017.json"
val_coco = "/content/coco2017/annotations/instances_val2017.json"

# batch size
train_batch_size = 8

# params for dataloader
train_shuffle_dl = True
val_shuffle_dl = False
test_shuffle_dl = False

# params for training

# two classes; only target class and background
num_classes = 81
num_epochs = 2

lr = 0.005
momentum = 0.9
weight_decay = 0.005

In [ ]:
class LitDataModule(pl.LightningDataModule):
  def __init__(self):
    super().__init__()
    self.save_hyperparameters()
    #self.transform = self.get_transform()
    self.transform = transforms.ToTensor()

  def get_transform(self):
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return transforms.Compose(custom_transforms)

  def setup(self, stage=None):
    self.train_dataset = myOwnDataset(root=train_data_dir, annotation=train_coco, transforms=self.transform)
    self.val_dataset = myOwnDataset(root=val_data_dir, annotation=val_coco, transforms=self.transform)   

  def train_dataloader(self):
    return DataLoader(self.train_dataset,
                      batch_size=train_batch_size,
                      shuffle=train_shuffle_dl,
                      collate_fn=collate_fn)
    
  """def val_dataloader(self):
    return DataLoader(self.val_dataset,
                      batch_size=train_batch_size,
                      shuffle=val_shuffle_dl,
                      collate_fn=collate_fn)"""
  


In [ ]:
class LitModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = get_model_instance_segmentation(81)
    
  def forward(self, x):
    x = self.model(x)
    return x

  def training_step(self, batch, batch_idx):
    imgs, annotations = batch
    loss_dict = self.model(imgs, annotations)
    loss = sum(loss for loss in loss_dict.values())
    self.log("train loss", loss)
    return loss

  def validation_step(self, batch, batch_idx):
    imgs, annotations = batch
    loss_dict = self.model(imgs, annotations)
    loss = sum(loss for loss in loss_dict.values())
    self.log("val loss", loss)
     
  def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

In [ ]:
dm = LitDataModule()
dm.setup()
model = LitModel()
trainer = pl.Trainer(max_epochs=num_epochs,
                  accelerator="auto",
                  devices=1 if torch.cuda.is_available() else None,
                  callbacks=[LearningRateMonitor(logging_interval="step"),
                                TQDMProgressBar(refresh_rate=20)],
)
trainer.fit(model, dm)

loading annotations into memory...
Done (t=20.94s)
creating index...
index created!
loading annotations into memory...
Done (t=0.43s)
creating index...
index created!


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:119: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  category=PossibleUserWarning,


loading annotations into memory...
Done (t=15.62s)
creating index...
index created!
loading annotations into memory...
Done (t=0.50s)
creating index...
index created!


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 41.7 M
-------------------------------------
41.5 M    Trainable params
222 K     Non-trainable params
41.7 M    Total params
166.816   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

AssertionError: ignored